<a href="https://colab.research.google.com/github/Abhi23run/Python/blob/main/Langchain/langchain_output_parsers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packagin

In [ ]:
from typing import List
from pydantic import BaseModel, Field, validator

class CodeRisk(BaseModel):
    description: str = Field(description="A brief description of the security risk")
    severity: str = Field(description="The level of severity of the risk")
    recommendations: List[str] = Field(description="Recommended actions to mitigate the risk")

    @validator("severity")
    def severity_must_be_valid(cls, field):
        if field not in ["Low", "Medium", "High", "Critical"]:
            raise ValueError("Invalid severity level!")
        return field

from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

# Assuming 'code_snippet' contains the code to be analyzed
query = f"Analyze this code for security risks: {code_snippet}"

# Set up the PydanticOutputParser with the CodeRisk model
parser = PydanticOutputParser(model=CodeRisk)

# Query the LLM and parse the output
llm = OpenAI(model_name='text-davinci-003', temperature=0)
response = llm.query(query)
structured_response = parser.parse(response)

print(structured_response)


In [ ]:
from langchain.llms import HuggingFaceLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define the prompt template for the interaction
prompt_template = PromptTemplate(
    template="Analyze this code for security risks: {code_snippet}",
    input_variables=["code_snippet"]
)

# Initialize the HuggingFaceLLM with the model name of LLaMA or Mistral
llm = HuggingFaceLLM(model_name="EleutherAI/gpt-neo-2.7B")

# Create an LLMChain with the LLM and prompt template
llm_chain = LLMChain(llm=llm, prompt_template=prompt_template)

# Define the code snippet to analyze
code_snippet = """
def check_password(password):
    # A dummy password check function
    if password == "password123":
        return True
    return False
"""

# Use the LLMChain to analyze the code snippet
response = llm_chain.run(code_snippet=code_snippet)

print(response)

In [ ]:
import openai
from pydantic import BaseModel, Field

# Define your Pydantic schema for the expected function call response
class FunctionCallResponse(BaseModel):
    name: str = Field(..., description="The name of the person")
    age: int = Field(..., description="The age of the person")

# Define the class that will handle the OpenAI function call
class OpenAIFunctionCaller:
    def __init__(self, api_key: str, model: str):
        """
        Initialize the OpenAI function caller with the necessary API key and model.

        :param api_key: Your OpenAI API key
        :param model: The model to use for the function call (e.g., "gpt-3.5-turbo")
        """
        self.api_key = api_key
        self.model = model
        openai.api_key = self.api_key

    def query(self, prompt: str) -> FunctionCallResponse:
        """
        Send a query to the OpenAI model and return a structured response.

        :param prompt: The prompt to send to the model
        :return: A FunctionCallResponse object with the structured data
        """
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "This is a function call."},
                {"role": "user", "content": prompt}
            ],
            # Assuming the function call is set up correctly in the prompt
        )

        # Extract the function call response from the completion
        function_call_data = response['choices'][0]['message']['content']

        # Deserialize the JSON response into a Pydantic model
        function_call_response = FunctionCallResponse.parse_raw(function_call_data)

        return function_call_response

# Usage example
# Replace 'your-api-key' with your actual OpenAI API key
function_caller = OpenAIFunctionCaller(api_key='your-api-key', model='gpt-3.5-turbo')
response = function_caller.query("Extract the name and age from the text: 'Jason is 25 years old'")
print(response.name)  # Output: Jason
print(response.age)   # Output: 25
